<a href="https://colab.research.google.com/github/JacquelineCl/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Connecting to pgAdmin
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:13 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drive

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Vine-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Grocery_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
print(df)

DataFrame[marketplace: string, customer_id: int, review_id: string, product_id: string, product_parent: int, product_title: string, product_category: string, star_rating: int, helpful_votes: int, total_votes: int, vine: string, verified_purchase: string, review_headline: string, review_body: string, review_date: string]


Crating and cleaning Vine vs Not Vine program data frames

In [4]:
vine_df = df.select(df.review_id, df.star_rating, df.helpful_votes, df.total_votes, df.vine, df.verified_purchase)

In [5]:
type(vine_df)

pyspark.sql.dataframe.DataFrame

In [6]:
print(vine_df)

DataFrame[review_id: string, star_rating: int, helpful_votes: int, total_votes: int, vine: string, verified_purchase: string]


In [7]:
useful_votes_df = vine_df.filter(vine_df.total_votes>20)

In [8]:
print(useful_votes_df)

DataFrame[review_id: string, star_rating: int, helpful_votes: int, total_votes: int, vine: string, verified_purchase: string]


In [9]:
type(useful_votes_df)

pyspark.sql.dataframe.DataFrame

In [10]:
helpful_votes_df = useful_votes_df.filter((useful_votes_df.helpful_votes/useful_votes_df.total_votes) > .5)

In [11]:
vine_program_cleaned_df = helpful_votes_df.filter(helpful_votes_df.vine == "Y")

In [19]:
print(vine_program_cleaned_df)

DataFrame[review_id: string, star_rating: int, helpful_votes: int, total_votes: int, vine: string, verified_purchase: string]


In [18]:
vine_program_cleaned_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3MKO875WGIEBJ|          5|           34|         37|   Y|                N|
|R3OPNTK61FQ7MP|          5|           32|         37|   Y|                N|
| RMDRU8I773X5U|          4|           22|         25|   Y|                N|
| RR83RGWFEFZCL|          3|           61|         64|   Y|                N|
|R2TMZ7GV8SPIRV|          4|           23|         23|   Y|                N|
| RX88JLUIFA4Q2|          4|           84|         87|   Y|                Y|
| RV3AHTDRHDH7R|          5|          253|        258|   Y|                N|
|R1A76AK493C8P8|          5|           83|         90|   Y|                N|
|R3LOTAAVOZ9D1L|          3|           19|         26|   Y|                N|
|R1BRN47DX8UXA7|          5|          154|        159|   Y|     

In [13]:
not_vine_program_cleaned_df = helpful_votes_df.filter(helpful_votes_df.vine == "N")

In [21]:
print(not_vine_program_cleaned_df)

DataFrame[review_id: string, star_rating: int, helpful_votes: int, total_votes: int, vine: string, verified_purchase: string]


In [20]:
not_vine_program_cleaned_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1OAZUG90XPU2U|          5|          250|        274|   N|                Y|
|R23VPTUJ8I8NR5|          1|           28|         30|   N|                Y|
| R6SRJFJ5YH4UM|          5|           23|         26|   N|                Y|
|R3DOS07BIMP4DJ|          2|           32|         42|   N|                N|
|R30PGWX4HN1CFG|          1|           85|         95|   N|                Y|
|R2XASGYIENUHQO|          5|           28|         32|   N|                Y|
|R34UQPQCTSKQK9|          5|           23|         26|   N|                Y|
|R2M6WNM2OZZHKS|          1|           20|         22|   N|                N|
| RO6JNMUJDE1LP|          5|           37|         50|   N|                N|
|R1WNL4OIMSKDBR|          5|           35|         36|   N|     

Vine Program Results

In [38]:
vine_program_total_reviews = vine_program_cleaned_df.agg({"total_votes": "sum"}).withColumnRenamed("sum(total_votes)", "total_votes")

In [51]:
vine_program_total_reviews.show()

+-----------+
|total_votes|
+-----------+
|       2829|
+-----------+



In [52]:
print(vine_program_total_reviews)

DataFrame[total_votes: bigint]


In [43]:
vine_program_5_star_reviews = vine_program_cleaned_df.filter(vine_program_cleaned_df.star_rating == 5).agg({"total_votes": "sum"}).withColumnRenamed("sum(total_votes)", "total_5_star_votes")

In [53]:
vine_program_5_star_reviews.show()

+------------------+
|total_5_star_votes|
+------------------+
|              1044|
+------------------+



In [54]:
vine_program_5_star_reviews

DataFrame[total_5_star_votes: bigint]

In [57]:
best_star_paid_percent = vine_program_5_star_reviews.total_5_star_votes/vine_program_total_reviews.total_votes

In [46]:
print(best_star_paid_percent)

Column<'(total_5_star_votes / total_votes)'>


Not Vine Program Results

In [62]:
not_vine_program_total_reviews = not_vine_program_cleaned_df.agg({"total_votes": "sum"}).withColumnRenamed("sum(total_votes)", "total_votes")

In [63]:
not_vine_program_total_reviews.show()

+-----------+
|total_votes|
+-----------+
|    1574578|
+-----------+



In [64]:
print(not_vine_program_total_reviews)

DataFrame[total_votes: bigint]


In [65]:
not_vine_program_5_star_reviews = not_vine_program_cleaned_df.filter(not_vine_program_cleaned_df.star_rating == 5).agg({"total_votes": "sum"}).withColumnRenamed("sum(total_votes)", "total_5_star_votes")

In [66]:
not_vine_program_5_star_reviews.show()

+------------------+
|total_5_star_votes|
+------------------+
|            895551|
+------------------+



In [67]:
not_vine_program_5_star_reviews

DataFrame[total_5_star_votes: bigint]

In [68]:
best_star_unpaid_percent = not_vine_program_5_star_reviews.total_5_star_votes/not_vine_program_total_reviews.total_votes

In [69]:
print(best_star_unpaid_percent)

Column<'(total_5_star_votes / total_votes)'>
